In [1]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import load_model

Using TensorFlow backend.


## Load Data

In [2]:
with open("./Files/notes.pkl", 'rb') as f:
    notes= pickle.load(f)

In [3]:
n_vocab = len(set(notes))

In [4]:
# ALl the unique elements in a sorted manner
pitchnames = sorted(set(item for item in notes))

In [5]:
# Mapping between note to int value
ele_to_int = dict((note, number) for number, note in enumerate(pitchnames))

## Load Model

In [6]:

model = load_model("./Model/weights-improvement-41-0.8011-bigger.hdf5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



# Predictions

In [7]:
sequence_length = 100
network_input = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [8]:
# Any random start index
start = np.random.randint(len(network_input) - 1)

# Mapping int_to_ele
int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))

In [9]:
# Initial pattern 
pattern = network_input[start]
prediction_output = []

# generate 200 elements
for note_index in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # convert into numpy desired shape 
    prediction_input = prediction_input/float(n_vocab) # normalise
    
    prediction =  model.predict(prediction_input, verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result) 
    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    pattern.append(idx)
    pattern = pattern[1:]

In [10]:
prediction_output

['E3',
 '9+0+4',
 'E3',
 'E3',
 'D3',
 '0+4',
 'C3',
 'E3',
 '0+2+7',
 'F3',
 '0+4+7',
 'E3',
 '4+7+9+0',
 'F2',
 'C3',
 '5+9+0',
 'G3',
 'C3',
 'F3',
 'C4',
 'C3',
 'F3',
 'A3',
 'F3',
 'C3',
 'E3',
 'A3',
 'E3',
 'D3',
 'C3',
 'G3',
 '2+5',
 'D3',
 'D3',
 'D3',
 'F3',
 '5+9',
 'D3',
 'F3',
 'D3',
 'F3',
 '2',
 'G2',
 'D3',
 'D3',
 'E3',
 'A4',
 'G3',
 'A4',
 'F3',
 'G3',
 'D3',
 'G3',
 '0+4',
 'D4',
 'D3',
 'D3',
 'D3',
 'D3',
 'G3',
 'D3',
 'E4',
 'D3',
 'G3',
 'C4',
 '0+4',
 'E4',
 'E3',
 'A4',
 'D3',
 'E3',
 'D4',
 'B4',
 'F#4',
 'D3',
 'F3',
 'C5',
 '0+4',
 'E4',
 'E3',
 'C5',
 'G4',
 'G4',
 'B4',
 'G4',
 'B-4',
 'G3',
 'G3',
 'C5',
 '2+7',
 'E4',
 'A4',
 'D3',
 'F3',
 'G4',
 'D3',
 'F3',
 'F4',
 'D3',
 'F3',
 'E4',
 '0+4',
 'E4',
 'G3',
 'F4',
 'G4',
 'E3',
 'G3',
 'E3',
 'G4',
 'G3',
 'A4',
 'G4',
 'A4',
 'G3',
 'C5',
 'G3',
 'C5',
 'C5',
 'B4',
 'G3',
 'C5',
 '0+4',
 'D5',
 'E3',
 'D5',
 'D5',
 'E3',
 'G3',
 'E3',
 'E3',
 'E3',
 'G3',
 'C5',
 '0+4',
 'F3',
 'E3',
 'C5',
 'D3',

## Create Midi File

In [11]:
offset = 0 # Time
output_notes = []

for pattern in prediction_output:
    
    # if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))  # create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        
        new_chord = chord.Chord(temp_notes) # creates the chord() from the list of notes
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    else:
            # if the pattern is a note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    
    ## can change according to listening music but giving equal to everyone
    offset += 0.5

In [12]:
print(output_notes)

[<music21.note.Note E>, <music21.chord.Chord A C E>, <music21.note.Note E>, <music21.note.Note E>, <music21.note.Note D>, <music21.chord.Chord C E>, <music21.note.Note C>, <music21.note.Note E>, <music21.chord.Chord C D G>, <music21.note.Note F>, <music21.chord.Chord C E G>, <music21.note.Note E>, <music21.chord.Chord E G A C>, <music21.note.Note F>, <music21.note.Note C>, <music21.chord.Chord F A C>, <music21.note.Note G>, <music21.note.Note C>, <music21.note.Note F>, <music21.note.Note C>, <music21.note.Note C>, <music21.note.Note F>, <music21.note.Note A>, <music21.note.Note F>, <music21.note.Note C>, <music21.note.Note E>, <music21.note.Note A>, <music21.note.Note E>, <music21.note.Note D>, <music21.note.Note C>, <music21.note.Note G>, <music21.chord.Chord D F>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note F>, <music21.chord.Chord F A>, <music21.note.Note D>, <music21.note.Note F>, <music21.note.Note D>, <music21.note.Note F>, <music21.cho

In [13]:
# create a stream object from the generated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "./Output Music/test_output.mid")

'./Output Music/test_output.mid'